In [1]:
import os

import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import math
import pickle


In [2]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [3]:
def get_scardec_stf(scardec_name, wanted_type = 'fctmoy'):
    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    time = np.array(time)
    return momentrate, time

In [4]:
def get_usgs_stf(usgs_name):
    data_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/USGS/'
    momentrate = []
    time = []

    with open(data_path + str(usgs_name), 'r') as f:
        data = f.readlines()
        for line in data:
            line = line.strip()
            line = line.rstrip()
            if line[0] not in ['0','1','2','3','4','5','6','7','8','9']:
                continue
            line = line.split()
            time.append(float(line[0]))
            momentrate.append(float(line[1]))

    momentrate = np.array(momentrate)
    time = np.array(time)

    momentrate = momentrate / 10**7 # convert to Nm from dyne cm
    return momentrate, time

In [5]:
def get_isc_stf(isc_name):
    isc_save_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/isc/'
    with open(f'{isc_save_path}{isc_name}/{isc_name}.txt', 'rb') as f:
        stf_list = pickle.load(f)
    with open(f'{isc_save_path}{isc_name}/{isc_name}_norm_info.txt', 'rb') as f:
        norm_dict = pickle.load(f)

    time = np.arange(0, 25.6, 0.1)
    momentrate = np.array(stf_list)*norm_dict['mo_norm']*10**8,
    return momentrate[0], time

In [6]:
def get_sigloch_stf(sigloch_name):
    data_path = '/home/siglochnas1/shared/AmplitudeProjects/pdata_processed/psdata_events/'
    momentrate = []
    time = []

    file_path = data_path + str(sigloch_name) + '/outfiles/ampinv.stf.xy'

    with open(file_path, 'r') as file:
        content = file.read()
        content = content.split('\n')
        greater_than_count = content.count('>')
        if greater_than_count > 0:
            time = [list(np.arange(0, 25.6, 0.1))]
            momentrate = [[]]
            for i in range(greater_than_count-1):
                time.append(list(np.arange(0, 25.6, 0.1)))
                momentrate.append([])


        stf_count = 0
        for c in content:
            if c not in ['<', '>', '']:
                split = c.split()
                #time[stf_count].append(float(split[0]))
                momentrate[stf_count].append(10**float(split[1]))
            else:
                stf_count += 1

    # time = np.arange(0, 25.6, 0.1)
    # time = np.array(time)
    return momentrate, time

In [7]:
def get_ye_stf(ye_name):
    data_path = '/home/earthquakes1/homes/Rebecca/phd/stf/data/Ye_et_al_2016/'
    momentrate = []
    time = []

    with open(data_path + str(ye_name), 'r') as f:
        data = f.readlines()
        for line in data:
            line = line.strip()
            line = line.rstrip()
            if line[0] not in ['0','1','2','3','4','5','6','7','8','9']:
                continue
            line = line.split()
            time.append(float(line[0]))
            momentrate.append(float(line[1]))
    momentrate = np.array(momentrate)
    time = np.array(time)
    return momentrate, time

In [8]:
def find_end_stf(momentrate, time, dataset = ''):
    not_zero = np.where(momentrate > 0)[0]
    #print(max(momentrate))
    start = min(not_zero)
    end = max(not_zero)

    detected_end = end
    detected_end_time = time[end]

    time = time[:end]
    momentrate = momentrate[:end]

    less_than_10 = np.where(momentrate <= 10*max(momentrate)/100)[0]

    if dataset == 'sigloch':
        start = np.where(momentrate > 0.05 * max(momentrate))[0][0]
    else:
        start = min(not_zero)
    #print(less_than_10)
    total_moment = scipy.integrate.simpson(momentrate[start:end],
                                        dx = time[1]-time[0])
    #print(less_than_10)
    for i in less_than_10:
        if i <= start:
            continue
        if i == 0:
            continue
        moment = scipy.integrate.simpson(momentrate[start:i],
                                        dx = time[1]-time[0])
        #print(i, moment/total_moment)
        if moment >= 0.5 * total_moment:
            #print('inif')
            #print(f'first time where < 10% of total momentrate and 50% of moment released: {time[i]} s')
            detected_end_time = time[i]
            detected_end = i
            #print(f'proportion of moment released: {(moment/total_moment)*100:.2f}%')
            break
    return detected_end_time, detected_end, time[start], start
    #return time[end], end

In [9]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):
    dx = time_opt[1]-time_opt[0]
    #print('dx', dx)
    #print('endtime', end_time)
    #print('points before zero', points_before_zero)
    end_window = (end_time/dx)+points_before_zero
    #print('end winodw', end_window)
    end = int(np.floor(end_window))
    if start == end:
        end += 1
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [10]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_scardec_ye_usgs_sigloch_isc_mag.csv')
combined.columns = ['event', 'scardec', 'ye', 'isc', 'sigloch', 'usgs', 'mag']

In [12]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

to_ignore = ['20051203_1610_1', '20071226_2204_2', '20030122_0206_1', '20090929_1748_0', '20120421_0125_1', '20110311_2011_2']

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = [[]]
all_relative_root_times_for_event = {}

for dataset, get_stf in zip(['scardec', 'ye', 'usgs', 'sigloch', 'isc'], [get_scardec_stf, get_ye_stf, get_usgs_stf, get_sigloch_stf, get_isc_stf]):
#for dataset, get_stf in zip(['sigloch'], [get_sigloch_stf]):
    print(dataset)
    all_durations = []
    all_root_times = []
    all_moments = []
    all_relative_root_times = [[]]
    all_relative_root_times_for_event = {}


    for proportion in proportions_list:
        value_error = 0
        print(proportion)
        durations = []
        root_times = []
        relative_root_times = []

        diff = []
        moments = []

        for i, row in combined.iterrows():

            name = row[dataset]


            if name == '0' or name == 0:
                continue
            #print(scardec_name)
            momentrate, time = get_stf(name)

            if dataset != 'sigloch':
                momentrate_list = [momentrate]
                time_list = [time]
            else:
                momentrate_list = momentrate
                time_list = time

            count = 0
            for momentrate, time in zip(momentrate_list, time_list):
                if time[0] == time[1]:
                    time = time[1:]

                if dataset != 'sigloch':
                    save_key = row.event
                else:
                    save_key = row.event + '_' + str(count)

                if save_key in to_ignore:
                    continue
                #print(time[0:10])
                
                momentrate = np.array(momentrate)
                #print(momentrate[0:1], momentrate[-2:-1])
                time = np.array(time)
                detected_end_time, detected_end, detected_start_time, detected_start = find_end_stf(momentrate, time, dataset)
                #print(detected_start_time)
                time = time - detected_start_time # shift to start STF at zero
                momentrate = momentrate[detected_start:]
                #print(time[0:10])
                detected_end_time = detected_end_time - detected_start_time

                not_zero = np.where(momentrate > 0)[0]

                dx = time[1]-time[0]


                start = 0 #detected_start # min(not_zero)
                end = detected_end - detected_start # max(not_zero)

                print(start, end)
                points_before_zero = abs(min(time)/dx)
                # print(time[0:10])
                # print(dx)
                # print(points_before_zero)

                duration = detected_end_time
                durations.append(duration)

                start_time = time[start]
                end_time = detected_end_time

                total_moment = scipy.integrate.simpson(momentrate[start:end], dx = dx)
                moments.append(total_moment)
                try:
                    root, r = scipy.optimize.bisect(f3,
                                                    start_time,
                                                    end_time,
                                                    rtol = 1e-6,
                                                    full_output = True,
                                                    args = (total_moment,
                                                            time,
                                                            momentrate,
                                                            start,
                                                            points_before_zero,
                                                            proportion,))
                except ValueError:
                    #print('value error')
                    #print(name)
                    durations = durations[:-1]
                    moments = moments[:-1]
                    value_error += 1
                    continue
                   
                root_idx = np.floor(root/dx)
                root_time = root_idx*dx
                root_times.append(root_time)
                relative_root_times.append(root_time-start_time)


                if save_key in all_relative_root_times_for_event.keys():
                    to_save = all_relative_root_times_for_event[save_key]
                    to_save.append((root_time-start_time)/duration)
                    all_relative_root_times_for_event[save_key] = to_save
                else:
                    all_relative_root_times_for_event[save_key] = [(root_time-start_time)/duration]

                if root_time-start_time > duration:
                    print('root time greater than duration, proportion:', proportion)
                    print(name)
                count += 1

        root_times = np.array(root_times)
        #print(root_times)
        durations = np.array(durations)
        moments = np.log10(np.array(moments))
        relative_root_times = np.array(relative_root_times)

        rel_root_times = relative_root_times/durations
        all_root_times.append(root_times[0])
        all_relative_root_times.append(rel_root_times)

        print(value_error)

    print(all_relative_root_times_for_event)

    df = pd.DataFrame.from_dict(all_relative_root_times_for_event, orient='index', columns=['10', '20', '30', '40', '50', '60', '70', '80', '90'])

    df.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/{dataset}_relative_root_times.csv')


scardec
0.1
0 302
0 178
0 69
0 100
0 295
0 115
0 215
0 179
0 78
0 88
0 57
0 102
0 139
0 295
0 58
0 59
0 121
0 246
0 168
0 139
0 45
0 144
0 36
0 97
0 104
0 44
0 134
0 150
0 140
0 55
0 81
0 28
0 164
0 45
0 335
0 70
0 127
0 110
0 82
0 199
0 256
0 115
0 114
0 165
0 66
0 194
0 136
0 57
0 125
0 117
0 94
0 123
0 181
0 276
0 295
0 564
0 70
0 183
0 100
0 88
0 211
0 162
0 64
0 232
0 165
0 95
0 99
0 50
0 85
0 231
0 69
0 96
0 85
0 50
0 259
0 72
0 56
0 79
0 171
0 111
0 97
0 42
0 90
0 93
0 57
0 232
0 64
0 151
0 96
0 54
0 125
0 189
0 101
0 136
0 69
0 62
0 96
0 98
0 162
0 154
0 118
0 59
0 73
0 106
0 83
0 64
0 77
0 62
0 101
0 69
0 347
0 56
0 87
0 85
0 51
0 210
0 130
0 41
0 161
0 162
0 145
0 58
0 121
0 84
0 38
0 146
0 51
0 156
0 100
0 311
0 152
0 102
0 324
0 148
0 144
0 354
0 106
0 149
0 66
0 99
0 68
0 260
0 157
0 123
0 69
0 112
0 48
0 73
0 80
0 122
0 55
0 224
0 146
0 204
0 63
0 63
0 65
0 153
0 105
0 230
0 97
0 125
0 36
0 117
0 93
0 147
0 65
0 61
0 42
0 82
0 121
0 88
0 145
0 108
0 42
0 45
0 36
0 226
0 1